# Climate Partner Impact
https://www.climateimpact.com

In [12]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

### Load data

In [13]:
with open('../input/ClimatePartnerImpact/projects_data.json', 'r') as file:
    data = json.load(file)
data[0]

{'id': 'omper396lw3gxwq0',
 'type': 'Feature',
 'geometry_type': 'Point',
 'geometry_coordinates': [[25.2555, -32.8334]],
 'properties_id': 'omper396lw3gxwq0',
 'properties_name': 'Somerset East Spekboom Restoration',
 'properties_slug': 'cpi2301',
 'properties_type': 'project',
 'properties_image': 'YFs3KVkT5wyLQp2XALcDPjv9MuhpUa4sPmlQbUXo.jpg',
 'properties_region': 'Eastern Cape',
 'properties_countries': [['South Africa']],
 'properties_activity_type': [['Restoration']]}

In [14]:
if isinstance(data, list) and isinstance(data[0], dict):
    df = pd.DataFrame(data)
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        7 non-null      object
 1   type                      7 non-null      object
 2   geometry_type             7 non-null      object
 3   geometry_coordinates      7 non-null      object
 4   properties_id             7 non-null      object
 5   properties_name           7 non-null      object
 6   properties_slug           7 non-null      object
 7   properties_type           7 non-null      object
 8   properties_image          7 non-null      object
 9   properties_region         7 non-null      object
 10  properties_countries      7 non-null      object
 11  properties_activity_type  7 non-null      object
dtypes: object(12)
memory usage: 804.0+ bytes


In [15]:
df.head(1)

,id,type,geometry_type,geometry_coordinates,properties_id,properties_name,properties_slug,properties_type,properties_image,properties_region,properties_countries,properties_activity_type
0,omper396lw3gxwq0,Feature,Point,"[[25.2555, -32.8334]]",omper396lw3gxwq0,Somerset East Spekboom Restoration,cpi2301,project,YFs3KVkT5wyLQp2XALcDPjv9MuhpUa4sPmlQbUXo.jpg,Eastern Cape,[[South Africa]],[[Restoration]]


### Fix geometries

In [16]:
df['geometry'] = df['geometry_coordinates'].apply(lambda x: Point(x[0]))

In [17]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'EPSG:4326')
gdf['geometry'] = gdf['geometry'].make_valid()

### Harmonize nomenclature

In [18]:
gdf['country'] = gdf['properties_countries'].apply(lambda x: x[0][0])
gdf = gdf[['id', 'properties_id', 'country', 'geometry']]

In [19]:
gdf['url'] = 'https://www.climateimpact.com'
gdf['host_name'] = 'Climate Partner Impact'

In [20]:
invalid_geom = False

if len(gdf[gdf.geometry.is_valid == False]) > 0:
    invalid_geom = True  
gdf['project_geometries_invalid'] = invalid_geom

In [21]:
columns_rename_mapping = {
    'id': 'site_id_reported',
    'properties_id': 'project_id_reported'
}
gdf.rename(columns=columns_rename_mapping, inplace=True)

In [22]:
gdf = gdf.assign(project_description_reported=None,site_sqkm=None,species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,planting_date_reported=None,country=None,site_id_reported=None,)
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   site_id_reported              0 non-null      object  
 1   project_id_reported           7 non-null      object  
 2   country                       0 non-null      object  
 3   geometry                      7 non-null      geometry
 4   url                           7 non-null      object  
 5   host_name                     7 non-null      object  
 6   project_geometries_invalid    7 non-null      bool    
 7   project_description_reported  0 non-null      object  
 8   site_sqkm                     0 non-null      object  
 9   species_count_reported        0 non-null      object  
 10  species_planted_reported      0 non-null      object  
 11  survival_rate_reported        0 non-null      object  
 12  trees_planted_reported        0 non-null      

In [23]:
gdf["planting_date_type"]= " "


### Save it

In [24]:
gdf.to_file('../midsave/climate_partner_impact.gpkg')